In [1]:
!pip install "stable-baselines3[extra]>=2.0.0a4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=3908d5113c59e82a9343df45fb894bc3cc5bfec12582cf1d0068d16e657519a9
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [2]:
def point_in_polygon(point, polygon):
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        x_inters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= x_inters:
                            inside = not inside
        p1x, p1y = p2x, p2y
    return inside

In [3]:
import numpy as np
import math
import gymnasium as gym
from gymnasium import spaces

class GolfEnv(gym.Env):
  def __init__(self):
    #Define the size of the grid
    self.grid_width = 200
    self.grid_height = 300
    self.shot_counter = 0

    #Define the action space as a Box space
    self.action_space = spaces.Box(
        low=np.array([0, 0, 0]),  # Minimum values for direction, club-type and swing strength #######
        high=np.array([180, 12, 3]),  # Maximum values for direction, club-type and swing strength ########
        dtype=np.float32
    )

    #Define the observation space
    self.observation_space = spaces.Dict({
          'x':spaces.Discrete(self.grid_width),
          'y':spaces.Discrete(self.grid_height)
      })

    #ball position
    self.ball_position = {
      'x': 50,
      'y': 0,
    }
    #green position
    self.green_left_corner = [self.grid_width-20, self.grid_height-20]
    self.green_right_corner = [self.grid_width, self.grid_height]

    #creating a dictionary to convert club to distance
    self.club_distances = {
            0: 200,
            1: 180,
            2: 170,
            3: 160,
            4: 150,
            5: 145,
            6: 138,
            7: 127,
            8: 120,
            9: 110,
            10: 97,
            11: 85,
            12: 55,
        }

    self.swing_strength = { #######
            0: 0.25,
            1: 0.5,
            2: 0.75,
            3: 1,
         }


  def step(self, action):

    # Extract direction, club type, and swing strength from the action
    direction = action[0]
    club_type = int(action[1])
    swing_strength = int(action[2])
    print(swing_strength)

    # Use direction and scaled distance by swing strength to calculate the new position
    angle_rad = math.radians(direction)
    scaled_distance = self.club_distances[club_type] * self.swing_strength[swing_strength]  # Scale distance by swing strength ########
    delta_x = scaled_distance * math.cos(angle_rad)
    delta_y = scaled_distance * math.sin(angle_rad)

    # Calculate new position
    new_x = self.ball_position['x'] + delta_x
    new_y = self.ball_position['y'] + delta_y

    new_x = round(new_x)
    new_y = round(new_y)

    #sets the reward to 0
    reward = 0
    terminated=False
    truncated=False
    #checks if the ball position is in the observation space and if not it sets it to the closest edge
    if new_x>self.observation_space['x'].n:
      reward = reward-10
      new_x=self.observation_space['x'].n
    if new_x<0:
      reward = reward-10
      new_x=0
    if new_y>self.observation_space['y'].n:
      reward = reward-10
      new_y=self.observation_space['y'].n

    #sets the ball position to the new position
    self.ball_position['x']=new_x
    self.ball_position['y']=new_y

    #checks if the ball position is on the green
    if (self.green_left_corner[0]<=self.ball_position['x']>=self.green_right_corner[0]) and (self.green_left_corner[1]<=self.ball_position['y']>=self.green_right_corner[1]):
      reward=10
      terminated = True

    bunker = [(25, 53), (26, 42), (36, 34), (49, 33), (56, 44), (78, 95), (86, 138), (86, 45), (78, 147), (67, 142), (62, 148), (60, 176), (42, 186), (31, 176), (29, 168), (30, 150), (57, 180), (58, 111), (29, 60)]

    ptc = (self.ball_position['x'], self.ball_position['y'])

    if point_in_polygon(ptc, bunker):
      reward=10

    #adds a shot to the shot counter
    self.shot_counter+=1

    #limit to 10 shots per round
    if self.shot_counter>=10:
      truncated=True

    observation = {
        'x': self.ball_position['x'],
        'y': self.ball_position['y']
    }

    return observation, reward, terminated, truncated, {}

  def reset(self, seed=None, options=None):
    #sets ball position to the middle of the start of the hole
    self.ball_position = {
        'x': 50,
        'y': 0,
    }

    observation = {
        'x': self.ball_position['x'],
        'y': self.ball_position['y']
    }

    return observation, {}









In [4]:
from stable_baselines3.common.env_checker import check_env

In [6]:
env = GolfEnv()
check_env(env, warn=True)

2
2
0
0
1
0
0
2
0
0
1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:441: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [57]:
env = GolfEnv()
observation = env.reset()
action=np.array([90,12,3])

observation, reward, done, truncated, info = env.step(action)

print(observation, reward)

3
{'x': 50, 'y': 55} 10


Train model using DDPG

In [11]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [59]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MultiInputPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=1000)
model.save("ddpg_golf")
vec_env = model.get_env()

del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_golf")

obs = vec_env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
1
0
2
0
2
1
2
1
1
2
0
2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3        |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 794      |
|    time_elapsed    | 0        |
|    total_timesteps | 12       |
---------------------------------
1
0
2
0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2        |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 709      |
|    time_elapsed    | 0        |
|    total_timesteps | 16       |
---------------------------------
1
1
1
1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.67     |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes   

RuntimeError: Class values must be smaller than num_classes.

In [61]:
obs = vec_env.reset()

action, _ = model.predict(obs, deterministic=True)

print(action)

[[88.75514    5.7872767  1.530386 ]]
